<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>

In this notebook the focus is on the calculation of Contractual Service Margin, Loss Component and Loss Recovery Component. 
<br><br>
The Contractual Service Margin ($CSM$) denotes the unearned profit from an insurance contract or group of insurance contracts and plays a critical role in the calculation of profit & loss values. Similarly, the unearned loss is denoted by Loss Component ($LC$), Loss Recovery Component in the case of reinsurance contracts.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "3ImportScope-Actuals"

# Experience Adjustment on Premium

<a id="experience-adjustment-on-premium"></a>
The Experience Adjustment (EA) on Premiums weights the cash-flow (CF aocSteps) for premium amount types by the PremiumAllocationFactor input for each group of insurance contract. 
The contributions of present values and actuals are computed separately.

$$
EA (\rm{PV}) = \text{Premium Allocation Factor} \cdot \big( PV (\text{AoC Type = CF}) \big)~, \\
$$
$$
EA (\text{Actual}) = \text{Premium Allocation Factor} \cdot \big( \text{Actual}(\text{AoC Type = CF}) \big) ~,
$$
where amount type premium and its children are considered, novelties in-force and new business are considered for $PV$ whereas novelty combined is considered for Actual.  The allocation is always done in the finest granularity (novelty, line of business, ..) possible.

In [0]:
public interface BeExperienceAdjustmentForPremium : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<BeExperienceAdjustmentForPremium>(s => s
                .WithApplicability<DefaultValueBeExperienceAdjustmentForPremium>(x => x.Identity.AocType != AocTypes.CF));

    string EstimateType => EstimateTypes.BEPA;
    string EconomicBasis => EconomicBases.L;
    
    PresentValue[] ByAmountType => GetStorage().GetPremiums().Select(pr => 
                                       Multiply( GetStorage().GetPremiumAllocationFactor(Identity), 
                                                 GetScope<PresentValue>((Identity, pr, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)) ) ).ToArray();                            
}

public interface DefaultValueBeExperienceAdjustmentForPremium : BeExperienceAdjustmentForPremium
{
    PresentValue[] BeExperienceAdjustmentForPremium.ByAmountType => Enumerable.Empty<PresentValue>().ToArray();
}

In [0]:
public interface ActualExperienceAdjustmentOnPremium : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<ActualExperienceAdjustmentOnPremium>(s => s
                .WithApplicability<DefaultValueActualExperienceAdjustmentOnPremium>(x => x.Identity.AocType != AocTypes.CF));
    
    public ActualBase[] ByAmountTypeAndEstimateType => GetStorage().GetPremiums().Select(pr =>
                                        Multiply( GetStorage().GetPremiumAllocationFactor(Identity),
                                                GetScope<ActualBase>((Identity, pr, EstimateTypes.A, (int?)null))) ).ToArray();
}

public interface DefaultValueActualExperienceAdjustmentOnPremium : ActualExperienceAdjustmentOnPremium
{
    ActualBase[] ActualExperienceAdjustmentOnPremium.ByAmountTypeAndEstimateType => Enumerable.Empty<ActualBase>().ToArray();
}

<a id="technical-margin"></a>
# Technical Margin

For the computation of the $CSM$ or $LC$ component values for each step of the [AoC Step Structure](#aoc-step-structure), it is convenient to introduce the 
notion of technical margin $TM$. In the case of BBA methodology, the Locked-in interest rates discounting is applied to obtain the PV and RA values.

This can be summarized as follows:

$$
TM(s) = \left\{
\begin{array}{rl}
TM(\rm{EOP}) \text{ of the previous period} ~ & \text{if }s = \text{BOP and Novelty is In-Force}.
\\
0 ~ & \text{if }s = \text{CF}.
\\
IAF \cdot~\text{Aggregated}~TM(\text{IA}) ~ & \text{if }s = \text{IA and Novelty is In-Force}.
\\
\rm{Premiums} + \text{Attributable Expense and Commissions} + \text{Investment Claims} ~ & \text{if }s = \text{EA}.
\\
-AF \cdot~\text{Aggregated}~TM(\text{AM})~ & \text{if }s = \text{AM}.
\\
\text{PV}(\text{s})\bigg|_{\substack{\text{Non Attributable} \\ \text{Amount Types} \\ \text{excluded}}}  
+ \text{RA}(\text{s})~ & \text{otherwise} 
\end{array}
\right.
$$

where

$$
\text{Aggregated}~TM (\text{AoC step}) = \sum_{s\in\text{ previous AoC steps}} TM(s)~.
$$

and the Interest Accretion Factor ($IAF$) is given by

$$
IAF = \prod_{i=1}^{TS} \text{Interest}_i - 1
$$

where $\text{Interest}_i$ is the monthly interest factor obtained from the [Yield Curve](#yield-curves) and $TS$ is the Time-Step.

Finally, the Premiums, Attributable Expense and Commissions and Investment Claims terms are given by:

$$
\rm{Premiums} = \sum_{\text{Amount Type}\in\{\text{PR and its children}\}}
EA(\rm{PV}) - EA(\text{Actual}) ~,
$$

$$
\text{Attributable Expense and Commissions} = \sum_{\text{Amount Type}\in\{\rm{ACA}, \rm{AEA}\}}
\big(PV_{\text{Novelty = I}} + PV_{\text{Novelty = N}} \big) - \text{Actual}_{\text{Novelty=C}} ~,
$$

$$
\text{Investment Claims } = \sum_{\text{Amount Type}\in\{\text{ICO and its children}\}}
\big(PV_{\text{Novelty = I}} + PV_{\text{Novelty = N}} \big) -  \text{Actual}_{\text{Novelty=C}}   ~,
$$

where the AoC Step **CF** is implicit for all formulas, $PV$ is the [present value](#present-value) with Estimate Type **BE** (Best Estimate), and Actuals have Estimate Types **A** (see details [here](#actual-values)). Note that according to the applied sign convention, the sign of the CF AoC Step of Best Estimate Present Value and Actual is the same, accounting for the minus sign in the formula. 

In [0]:
public interface TechnicalMargin : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) => 
        builder.ForScope<TechnicalMargin>(s => s.WithApplicability<TechnicalMarginForCurrentBasis>(x => x.Identity.ValuationApproach == ValuationApproaches.VFA, p => p.ForMember(s => s.EconomicBasis))
                                               .WithApplicability<TechnicalMarginForBOP>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I)
                                               .WithApplicability<TechnicalMarginDefaultValue>(x => x.Identity.AocType == AocTypes.CF)
                                               .WithApplicability<TechnicalMarginForIA>(x => x.Identity.AocType == AocTypes.IA && x.Identity.Novelty == Novelties.I)
                                               .WithApplicability<TechnicalMarginForEA>(x => x.Identity.AocType == AocTypes.EA && !x.Identity.IsReinsurance)
                                               .WithApplicability<TechnicalMarginForAM>(x => x.Identity.AocType == AocTypes.AM)
                                               );
    
    [NotVisible] string EconomicBasis => EconomicBases.L;
    
    double Value => GetScope<ValidAmountType>(Identity.DataNode).BeAmountTypes
                       .Except(GetStorage().GetNonAttributableAmountType())
                       .Sum(at => GetScope<PresentValue>((Identity, at, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value) +
                       GetScope<PresentValue>((Identity, (string)null, EstimateTypes.RA, (int?)null), o => o.WithContext(EconomicBasis)).Value;
                    
    double AggregatedValue => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                                .Sum(aoc => GetScope<TechnicalMargin>(Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);
}

public interface TechnicalMarginForCurrentBasis : TechnicalMargin
{
    [NotVisible] string TechnicalMargin.EconomicBasis => EconomicBases.C;
}

public interface TechnicalMarginForBOP : TechnicalMargin
{
    private double ValueCsm => GetStorage().GetValue(Identity, null, EstimateTypes.C, null, Identity.ProjectionPeriod);
    private double ValueLc => GetStorage().GetValue(Identity, null, EstimateTypes.L, null, Identity.ProjectionPeriod);
    private double ValueLr => GetStorage().GetValue(Identity, null, EstimateTypes.LR, null, Identity.ProjectionPeriod);
    
    double TechnicalMargin.Value => -1d * ValueCsm + ValueLc + ValueLr;
}

public interface TechnicalMarginDefaultValue : TechnicalMargin
{
    double TechnicalMargin.Value => default;
}

public interface TechnicalMarginForIA : TechnicalMargin
{
    private int timeStep => GetStorage().GetTimeStep(Identity.ProjectionPeriod);
    private int shift => GetStorage().GetShift(Identity.ProjectionPeriod);
    
    private double[] monthlyInterestFactor => GetScope<MonthlyRate>(Identity, o => o.WithContext(EconomicBasis)).Interest;
    
    private double interestAccretionFactor => Enumerable.Range(shift,timeStep)
                                                    .Select(i => monthlyInterestFactor.GetValidElement(i/12))
                                                    .Aggregate(1d, (x, y) => x * y ) - 1d;
    
    double TechnicalMargin.Value => AggregatedValue * interestAccretionFactor;
}

public interface TechnicalMarginForEA : TechnicalMargin
{
    static ApplicabilityBuilder ScopeApplicabilityBuilderInner(ApplicabilityBuilder builder) => 
        builder.ForScope<TechnicalMargin>(s => s.WithApplicability<TechnicalMarginDefaultValue>(x => x.Identity.IsReinsurance));
                                               
    private string referenceAocType => GetScope<ReferenceAocStep>(Identity).Value.AocType;
    
    private double premiums => GetStorage().GetNovelties(referenceAocType, InputSource.Cashflow)
                                    .Sum(n => GetScope<BeExperienceAdjustmentForPremium>(Identity with {AocType = referenceAocType, Novelty = n}).ByAmountType.Sum(sc => sc.Value)) -
                                GetScope<ActualExperienceAdjustmentOnPremium>(Identity with {AocType = referenceAocType, Novelty = Novelties.C}).ByAmountTypeAndEstimateType.Sum(sc => sc.Value);
                           
    private double attributableExpenseAndCommissions => GetStorage().GetAttributableExpenseAndCommissionAmountType().Sum(d =>
                                                         GetStorage().GetNovelties(referenceAocType, InputSource.Cashflow)
                                                                     .Sum(n => GetScope<PresentValue>((Identity with {AocType = referenceAocType, Novelty = n}, d, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value) -
                                                         GetScope<ActualBase>((Identity with {AocType = referenceAocType, Novelty = Novelties.C}, d, EstimateTypes.A, (int?)null)).Value);

    private double investmentClaims => GetStorage().GetInvestmentClaims().Sum(ic =>
                                         GetStorage().GetNovelties(referenceAocType, InputSource.Cashflow)
                                                     .Sum(n => GetScope<PresentValue>((Identity with {AocType = referenceAocType, Novelty = n}, ic, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value) -
                                         GetScope<ActualBase>((Identity with {AocType = referenceAocType, Novelty = Novelties.C}, ic, EstimateTypes.A, (int?)null)).Value);
    
    double TechnicalMargin.Value => premiums + attributableExpenseAndCommissions + investmentClaims;
}

public interface TechnicalMarginForAM : TechnicalMargin
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<TechnicalMarginForAM>(s => s.WithApplicability<TechnicalMarginForAmForPaa>(x => x.Identity.ValuationApproach == ValuationApproaches.PAA));   
 
    double TechnicalMargin.Value => -1d * AggregatedValue * GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.CU, 0), o => o.WithContext(EconomicBasis)).Value;
}

public interface TechnicalMarginForAmForPaa : TechnicalMargin
{
    double TechnicalMargin.Value => GetScope<ValidAmountType>(Identity.DataNode).BeAmountTypes
                                   .Except(GetStorage().GetNonAttributableAmountType())
                                   .Sum(at => GetScope<PresentValue>((Identity with {AocType = AocTypes.CF, Novelty = Novelties.I}, at, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value +
                                              GetScope<PresentValue>((Identity with {AocType = AocTypes.CF, Novelty = Novelties.N}, at, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value ) +
                                    GetScope<PresentValue>((Identity with {AocType = AocTypes.CF, Novelty = Novelties.I}, (string)null, EstimateTypes.RA, (int?)null), o => o.WithContext(EconomicBasis)).Value +
                                    GetScope<PresentValue>((Identity with {AocType = AocTypes.CF, Novelty = Novelties.N}, (string)null, EstimateTypes.RA, (int?)null), o => o.WithContext(EconomicBasis)).Value;
}

# Switch Logic
    
The $CSM$ and $LC$ components are computed from the [technical margin](#technical-margin). In particular, for a given AoC step $s$, TM is allocated to the $LC$ when $\text{Aggregated } TM$ is positive whereas it is allocated to the $CSM$ when $\text{Aggregated } TM$ is negative:

$$
\begin{array}{rl}
CSM({\text{AoC Step}}) = 0,~~ LC({\text{AoC step}}) = TM({\text{AoC step}}) ~ & \text{if }\text{Aggregated }TM({\text{AoC step}}) > 0.
\\
CSM({\text{AoC Step}}) = - TM({\text{AoC step}}),~~ LC({\text{AoC step}}) = 0 ~ & \text{otherwise} 
\end{array}
$$


The figures reported under CSM are the opposite of the TM value in order to satisfy our sign convention.

It is possible to switch from $CSM$ to $LC$ and from $LC$ to $CSM$ at any AoC step $s$ with the only exception of **Amortization** where there is no switch from the previous step.
When a switch occurs the total contribution to the $CSM$ ($LC$) prior the switching step is brought to 0 and the remaing amount is allocated to $LC$ ($CSM$).




## Gross case

The switch logic is applied ***separately*** to the In-Force and New Business novelties. The Combined Liability AoC Step **CL** will bring both contributions to CSM and LC together as the novelities are summed up.

In detail, and as we go through the AoC steps in the AoC chain, we have

**A)** For the **BOP**:
$$
\begin{array}{rl}
CSM(\text{BOP}) &= CSM(\text{EOP}) \text{ of the previous period, for Novelty In-Force}
\\
LC(\text{BOP}) &= LC(\text{EOP}) \text{ of the previous period, for Novelty In-Force}
\end{array}
$$


**B)** For the **following AoC steps**, the switch logic is preferably formulated using the delta variations between steps, $\Delta CSM$ and $\Delta LC$ for the CSM and the LC respectively.

The switch logic for these AoC steps with the *exception* of Combined Liability is:

If $\text{Aggregated }TM(\text{AoC Step}) > 0$ and $\text{Aggregated }TM(\text{AoC Step}) + TM(\text{AoC Step}) \le 0$, then there is a switch from $LC$ to $CSM$:

$$
\begin{array}{rl}
\Delta CSM(\text{AoC step}) &= -TM(\text{AoC Step}) -\text{Aggregated } TM(\text{AoC Step})
\\
\Delta LC(\text{AoC step}) &= -\text{Aggregated } TM(\text{AoC Step}) 
\end{array}
$$

On the other hand, if $\text{Aggregated }TM(\text{AoC Step}) \le 0$ and $\text{Aggregated }TM(\text{AoC Step}) + TM(\text{AoC Step}) > 0$, then the switch is from $CSM$ to $LC$:

$$
\begin{array}{rl}
\Delta CSM(\text{AoC step}) &= \text{Aggregated } TM(\text{AoC Step}) 
\\
\Delta LC(\text{AoC step}) &= TM(\text{AoC Step}) + \text{Aggregated } TM(\text{AoC Step}) 
\end{array}
$$

If there is no switch from $CSM$, i.e. $\text{Aggregated }TM(\text{AoC Step}) \le 0$ and $\text{Aggregated }TM(\text{AoC Step}) + TM(\text{AoC Step}) \le 0 $, then:

$$
\begin{array}{rl}
\Delta CSM(\text{AoC step}) &= -TM(\text{AoC Step})
\\
\Delta LC(\text{AoC step}) &= 0
\end{array}
$$

And finally, if there is no switch from $LC$, i.e. $\text{Aggregated }TM(\text{AoC Step}) > 0$ and $\text{Aggregated }TM(\text{AoC Step}) + TM(\text{AoC Step}) \ge 0 $, then:

$$
\begin{array}{rl}
\Delta CSM(\text{AoC step}) &= 0
\\
\Delta LC(\text{AoC step}) &= TM(\text{AoC Step})
\end{array}
$$

**C)** For the AoC step Combined Liability **CL**, the switch logic is similar to the one above, except that a switch from $LC$ to $CSM$ can happen **because of New Business and despite of In-Force**, in which case we have:

$$
\begin{array}{rl}
\Delta CSM(\text{AoC step}) &= -TM(\text{AoC Step}) -\big(\text{Aggregated } TM(\text{Last In-Force AoC step})+TM(\text{Last In-Force AoC step}) \big)
\\
\Delta LC(\text{AoC step}) &= -\big(\text{Aggregated } TM(\text{Last In-Force AoC step}) + TM(\text{Last In-Force AoC step}) \big)
\end{array}
$$

If, on the other hand, the switch from $LC$ to $CSM$ happens **because of In-Force and despite of New Business**, then we have:

$$
\begin{array}{rl}
\Delta CSM(\text{AoC step}) &= -TM(\text{AoC Step}) -\big(\text{Aggregated } TM(\text{Last NB AoC step})+TM(\text{Last NB AoC step}) \big)
\\
\Delta LC(\text{AoC step}) &= -\big(\text{Aggregated } TM(\text{Last NB AoC step}) + TM(\text{Last NB AoC step}) \big)
\end{array}
$$

For the switch in the other direction, i.e. from $CSM$ to $LC$ the formulas are similar except that LC and CSM are swapped.

**D)** Finally, for the last AoC step, the **EOP**, we have:

$$
\begin{array}{rl}
CSM(\text{EOP}) &= \sum_{s~\in~\text{previous AoC steps}}\Delta CSM(s)
\\
LC(\text{EOP}) &= \sum_{s~\in~\text{previous AoC steps}}\Delta LC(s)
\end{array}
$$



In [0]:
public interface AllocateTechnicalMarginWithIfrsVariable: IScope<ImportIdentity, ImportStorage>
{                                  
    double Value => ComputeTechnicalMarginFromIfrsVariables(Identity);
    double AggregatedValue => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                               .Sum(aoc => ComputeTechnicalMarginFromIfrsVariables(Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty}));
                                                                    
    private double ComputeTechnicalMarginFromIfrsVariables(ImportIdentity id) =>
        GetStorage().GetValue(Identity, null, EstimateTypes.LR, null, Identity.ProjectionPeriod) +
        GetStorage().GetValue(Identity, null, EstimateTypes.L, null, Identity.ProjectionPeriod) - 
        GetStorage().GetValue(Identity, null, EstimateTypes.C, null, Identity.ProjectionPeriod);
}

In [0]:
public interface AllocateTechnicalMargin: IScope<ImportIdentity, ImportStorage>
{
    //Switch
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) => 
        builder.ForScope<AllocateTechnicalMargin>(s => s
                                             .WithApplicability<ComputeAllocateTechnicalMarginWithIfrsVariable>(x => x.GetStorage().IsSecondaryScope(x.Identity.DataNode))
                                             .WithApplicability<AllocateTechnicalMarginForBopProjection>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I && x.Identity.ProjectionPeriod > 0)
                                             .WithApplicability<AllocateTechnicalMarginForBop>(x => x.Identity.AocType == AocTypes.BOP)
                                             .WithApplicability<AllocateTechnicalMarginForCl>(x => x.Identity.AocType == AocTypes.CL)
                                             .WithApplicability<AllocateTechnicalMarginForEop>(x => x.Identity.AocType == AocTypes.EOP)
                                             );
    
    [NotVisible] double AggregatedTechnicalMargin => GetScope<TechnicalMargin>(Identity).AggregatedValue; 
    [NotVisible] double TechnicalMargin => GetScope<TechnicalMargin>(Identity).Value;
    [NotVisible] string ComputedEstimateType => ComputeEstimateType(GetScope<TechnicalMargin>(Identity).AggregatedValue + TechnicalMargin);
    [NotVisible] bool HasSwitch => ComputedEstimateType != ComputeEstimateType(GetScope<TechnicalMargin>(Identity).AggregatedValue);
     
    //Allocate
    [NotVisible] string EstimateType => GetContext();
    
    double Value => (HasSwitch, EstimateType == ComputedEstimateType) switch {
            (true, true) => TechnicalMargin + AggregatedTechnicalMargin,
            (true, false) => -1d * AggregatedTechnicalMargin,
            (false, true) => TechnicalMargin,
            _ => default
        };
    
    string ComputeEstimateType(double aggregatedTechnicalMargin) => aggregatedTechnicalMargin > Precision ? EstimateTypes.L : EstimateTypes.C;
}

public interface ComputeAllocateTechnicalMarginWithIfrsVariable : AllocateTechnicalMargin
{                                  
    double AllocateTechnicalMargin.TechnicalMargin => GetScope<AllocateTechnicalMarginWithIfrsVariable>(Identity).Value;
    double AllocateTechnicalMargin.AggregatedTechnicalMargin => GetScope<AllocateTechnicalMarginWithIfrsVariable>(Identity).AggregatedValue;
}

public interface AllocateTechnicalMarginForCl : AllocateTechnicalMargin
{
    private double balancingValue => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                                        .GroupBy(x => x.Novelty, (k, v) => v.Last())
                                        .Sum(aoc => { 
                                            var id = Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty};
                                            return GetScope<AllocateTechnicalMargin>(id).ComputedEstimateType != ComputedEstimateType ? 
                                                   GetScope<AllocateTechnicalMargin>(id).TechnicalMargin + GetScope<AllocateTechnicalMargin>(id).AggregatedTechnicalMargin
                                                   : (double)default; });

    [NotVisible] bool AllocateTechnicalMargin.HasSwitch => Math.Abs(balancingValue) > Precision;
    [NotVisible] double AllocateTechnicalMargin.AggregatedTechnicalMargin => balancingValue;
}

public interface AllocateTechnicalMarginForBop : AllocateTechnicalMargin {
    bool AllocateTechnicalMargin.HasSwitch => false;
}

public interface AllocateTechnicalMarginForEop : AllocateTechnicalMargin
{
    double AllocateTechnicalMargin.Value  => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                                                .Sum(aoc => GetScope<AllocateTechnicalMargin>(Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);
    [NotVisible] string AllocateTechnicalMargin.ComputedEstimateType => ComputeEstimateType(AggregatedTechnicalMargin);
}

public interface AllocateTechnicalMarginForBopProjection: AllocateTechnicalMargin{
    double AllocateTechnicalMargin.TechnicalMargin => GetScope<AllocateTechnicalMargin>(Identity with {AocType = AocTypes.EOP, Novelty = Novelties.C, ProjectionPeriod = Identity.ProjectionPeriod - 1}).Value;
    bool AllocateTechnicalMargin.HasSwitch => false;
}

# Contractual Service Margin (Gross)

The scope below is simply used to set the appropriate Estimate Type C for $CSM$:

In [0]:
public interface ContractualServiceMarginGross : IScope<ImportIdentity, ImportStorage>
{
    [NotVisible]string EstimateType => EstimateTypes.C;
    
    double Value => -1d * GetScope<AllocateTechnicalMargin>(Identity, o => o.WithContext(EstimateType)).Value;
}

# Loss Component

The scope below is simply used to set the appropriate Estimate Type L for $LC$ :

In [0]:
public interface LossComponent : IScope<ImportIdentity, ImportStorage>
{
    [NotVisible]string EstimateType => EstimateTypes.L;
    
    double Value => GetScope<AllocateTechnicalMargin>(Identity, o => o.WithContext(EstimateType)).Value;
}

# Technical Margin allocation for Reinsurance

In the case of Reinsurance a Loss Recovery Component is computed when the underlying gross business opens a Loss component. Loss Recovery Component uses the notion of Loss Recovery Component Boundary ([LoReCo Boundary](#loreco-boundary)). 

## LoReCo Boundary

The LoReCo Boundary is the underlying gross Loss Component multiplied by the weights coming from the Reinsurance Coverage (RC) Parameter (see [Data Node Parameters](../DataModel/DataStructure#data-node-parameters)). In case a GRIC ($r$) has multiple GICs ($g$), then the LoReCo Boundaries computed for each GIC-GRIC link are aggregated.

$$
\text{LoReCo Boundary}(\text{r}) = \sum_{g~\in~\text{underlying GICs}} \text{RC}_{g,r} \text{LC}(g).
$$

In [0]:
public interface LoReCoBoundary : IScope<ImportIdentity, ImportStorage>
{
    private IEnumerable<string> underlyingGic => GetStorage().GetUnderlyingGic(Identity, LiabilityTypes.LRC);
   
    double Value => underlyingGic.Sum(gic => GetStorage().GetReinsuranceCoverage(Identity, gic) * 
                                             GetScope<LossComponent>(Identity with {DataNode = gic}).Value);
                                                                      

    double AggregatedValue => underlyingGic.Sum(gic => GetStorage().GetReinsuranceCoverage(Identity, gic) * 
                                                       GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                                                            .Sum(aoc => GetScope<LossComponent>(Identity with {DataNode = gic, AocType = aoc.AocType, Novelty = aoc.Novelty}).Value));
}

## Reinsurance Switch Logic

For the **Group of Reinsurance Contracts** (GRICs), the switch is computed within the underlying GIC context and $CSM$ and $LR$ are then computed in a similar as for the underlying GICs CSM and LC. In the reinsurance case, the switch is controlled by the sign of the LoReCo Boundary rather than the reinsurance Technical Margin. This allows the reinsurance LoReCo allocation to be controlled by gross switch calculation. 
The amount allocated to the Reinsurance CSM or LoReCo follows the computed Technical Margin computed for GRIC r as described in [Technical Margin](#technical-margin).

For the , the switch is computed within the underlying GIC context and $CSM$ and $LR$ are then computed in the same manner as for the underlying GICs CSM and LC.

Within the **Premium Allocation Approach** the LoReCo Component corresponds to the computed [LoReCo Boundary](loreco-boundary).

## Contractual Service Margin (Reinsurance)

In [0]:
public interface CsmReinsurance: AllocateTechnicalMargin
{
    [NotVisible] string AllocateTechnicalMargin.EstimateType => EstimateTypes.C;
    [NotVisible] string AllocateTechnicalMargin.ComputedEstimateType => EstimateType;
    [NotVisible] bool AllocateTechnicalMargin.HasSwitch => false;

    //Switch
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) => 
        builder.ForScope<CsmReinsurance>(s => s
                                             .WithApplicability<AllocateTechnicalMarginForBopProjection>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I && x.Identity.ProjectionPeriod > 0)
                                             .WithApplicability<CsmReinsuranceEop>(x => x.Identity.AocType == AocTypes.EOP)
                                             .WithApplicability<AllocationForReinsurance>(x => true, //from here the applicabilities of the AllocateTechnicalMargin are applied with the following members redefined here.
                                                                                               p => p.ForMember(s => s.ComputedEstimateType)
                                                                                                     .ForMember(s => s.HasSwitch))
                                            );
}

public interface AllocationForReinsurance : CsmReinsurance{
    [NotVisible] string AllocateTechnicalMargin.ComputedEstimateType => EstimateType;
    [NotVisible] bool AllocateTechnicalMargin.HasSwitch => false;
}

public interface AllocateTechnicalMarginForBopProjection: AllocateTechnicalMargin{
    double AllocateTechnicalMargin.TechnicalMargin => GetScope<CsmReinsurance>(Identity with {AocType = AocTypes.EOP, Novelty = Novelties.C, ProjectionPeriod = Identity.ProjectionPeriod - 1}).Value;  
}

public interface CsmReinsuranceEop : CsmReinsurance{
    double AllocateTechnicalMargin.Value  => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                                                .Sum(aoc => GetScope<CsmReinsurance>(Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);
}

## Loss Recovery Component

In [0]:
public interface LorecoCalculation : IScope<ImportIdentity, ImportStorage>
{    
    //Switch
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) => 
        builder.ForScope<LorecoCalculation>(s => s
                                             .WithApplicability<LorecoCalculationForBopProjection>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I && x.Identity.ProjectionPeriod > 0)
                                             .WithApplicability<LorecoCalculationPaa>(x => x.Identity.ValuationApproach == ValuationApproaches.PAA)
                                             .WithApplicability<LorecoCalculationEop>(x => x.Identity.AocType == AocTypes.EOP)
                                            );

    protected double loReCoBoundaryValue => GetScope<LoReCoBoundary>(Identity).Value;
    protected double loReCoBoundaryAggregatedValue => GetScope<LoReCoBoundary>(Identity).AggregatedValue;

    double Value => -1d * loReCoBoundaryValue; //TODO fix here
}

public interface LorecoCalculationForBopProjection: LorecoCalculation{
    double LorecoCalculation.Value => GetScope<LorecoCalculation>(Identity with {AocType = AocTypes.EOP, Novelty = Novelties.C, ProjectionPeriod = Identity.ProjectionPeriod - 1}).Value;
}

public interface LorecoCalculationPaa : LorecoCalculation{
    double LorecoCalculation.Value => -1d * loReCoBoundaryValue;
}

public interface LorecoCalculationEop : LorecoCalculation{
    double LorecoCalculation.Value  => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                                                .Sum(aoc => GetScope<LorecoCalculation>(Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);
}

In [0]:
// public interface AllocateTechnicalMarginForReinsurance: AllocateTechnicalMargin
// {
//     //Switch
//     static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) => 
//         builder.ForScope<AllocateTechnicalMarginForReinsurance>(s => s
//                                              .WithApplicability<AllocateTechnicalMarginForBopProjection>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I && x.Identity.ProjectionPeriod > 0)
//                                              .WithApplicability<AllocateTechnicalMarginForReinsurancePaa>(x => x.Identity.ValuationApproach == ValuationApproaches.PAA)
//                                              .WithApplicability<AllocateTechnicalMarginForReinsuranceCL>(x => x.Identity.AocType == AocTypes.CL)
//                                              .WithApplicability<AllocateTechnicalMarginForReinsuranceEop>(x => x.Identity.AocType == AocTypes.EOP)
//                                              .WithApplicability<AllocationForReinsurance>(x => true, //from here the applicabilities of the AllocateTechnicalMargin are applied with the following members redefined here.
//                                                                                                p => p.ForMember(s => s.ComputedEstimateType)
//                                                                                                      .ForMember(s => s.HasSwitch))
//                                             );
//     protected string ComputeReinsuranceEstimateType(double aggregatedFcf) => aggregatedFcf > Precision ? EstimateTypes.LR : EstimateTypes.C;
//     protected double loReCoBoundaryValue => GetScope<LoReCoBoundary>(Identity).Value;
//     protected double loReCoBoundaryAggregatedValue => GetScope<LoReCoBoundary>(Identity).AggregatedValue;
// }

// public interface AllocationForReinsurance : AllocateTechnicalMarginForReinsurance{
//     [NotVisible] string AllocateTechnicalMargin.ComputedEstimateType => ComputeReinsuranceEstimateType(loReCoBoundaryAggregatedValue + loReCoBoundaryValue);
//     [NotVisible] bool AllocateTechnicalMargin.HasSwitch => ComputedEstimateType != ComputeReinsuranceEstimateType(loReCoBoundaryAggregatedValue);
// }

// public interface AllocateTechnicalMarginForReinsurancePaa : AllocateTechnicalMarginForReinsurance{
//     double AllocateTechnicalMargin.Value => EstimateType switch {
//         EstimateTypes.LR => -1d * GetScope<LoReCoBoundary>(Identity).Value,
//         _ => default
//     };
// }

// public interface AllocateTechnicalMarginForReinsuranceCL : AllocateTechnicalMarginForReinsurance{   
//     string AllocateTechnicalMargin.ComputedEstimateType => ComputeReinsuranceEstimateType(loReCoBoundaryAggregatedValue + loReCoBoundaryValue);
//     private double balancingValue => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
//                                         .GroupBy(x => x.Novelty, (k, v) => v.Last())
//                                         .Sum(aoc => { 
//                                             var id = Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty};
//                                             return GetScope<AllocateTechnicalMarginForReinsurance>(id).ComputedEstimateType != ComputedEstimateType ? 
//                                                    GetScope<AllocateTechnicalMarginForReinsurance>(id).TechnicalMargin + GetScope<AllocateTechnicalMarginForReinsurance>(id).AggregatedTechnicalMargin
//                                                    : (double)default; });
                                                   
//     [NotVisible] bool AllocateTechnicalMargin.HasSwitch => Math.Abs(balancingValue) > Precision;
//     [NotVisible] double AllocateTechnicalMargin.AggregatedTechnicalMargin => balancingValue;    
// }

// public interface AllocateTechnicalMarginForReinsuranceEop : AllocateTechnicalMarginForReinsurance{
//     double AllocateTechnicalMargin.Value  => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
//                                                 .Sum(aoc => GetScope<AllocateTechnicalMarginForReinsurance>(Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);
//     [NotVisible] string AllocateTechnicalMargin.ComputedEstimateType => ComputeReinsuranceEstimateType(AggregatedTechnicalMargin);
// }

// public interface AllocateTechnicalMarginForBopProjection: AllocateTechnicalMargin{
//     double AllocateTechnicalMargin.TechnicalMargin => GetScope<AllocateTechnicalMarginForReinsurance>(Identity with {AocType = AocTypes.EOP, Novelty = Novelties.C, ProjectionPeriod = Identity.ProjectionPeriod - 1}).Value;
//     bool AllocateTechnicalMargin.HasSwitch => false;
// }

# Loss Recovery Component

In [0]:
public interface LossRecoveryComponent : IScope<ImportIdentity, ImportStorage>
{
    [NotVisible]string EstimateType => EstimateTypes.LR;
    
    double Value => GetScope<LorecoCalculation>(Identity).Value;
}

# Contractual Service Margin

In [0]:
public interface ContractualServiceMargin : IScope<ImportIdentity, ImportStorage>
{
    [NotVisible]string EstimateType => EstimateTypes.C;
     
    double Value => Identity.IsReinsurance 
        ? -1d * GetScope<CsmReinsurance>(Identity, o => o.WithContext(EstimateType)).Value
        : GetScope<ContractualServiceMarginGross>(Identity, o => o.WithContext(EstimateType)).Value;
}
